In [ ]:
import warnings
warnings.simplefilter('always', DeprecationWarning)

from osgeo import gdal
from osgeo import ogr
from osgeo import osr

import fiona
import rasterio

import numpy as np
import math
# from osgeo import gdal_array
# from osgeo import gdalnumeric

import os
import contextlib
import logging

gdal.UseExceptions()

def silent_remove(path):
    with contextlib.suppress(FileNotFoundError):
        os.remove(path)
        
DEBUG=False
logging.basicConfig(level=logging.DEBUG if DEBUG else logging.ERROR)
logging.getLogger('fiona.ogrext').disabled=True # Too noisy
fiona.Env(CPL_DEBUG=DEBUG).__enter__()
rasterio.Env(CPL_DEBUG=DEBUG).__enter__()

# Lecture de l'ENC

In [ ]:
enc_path = 'data/ENC_ROOT/US4MA13M/US4MA13M.000'
m_covr_lyr = fiona.open(enc_path, layer='M_COVR', LIST_AS_STRING=True)
soundg_lyr = fiona.open(enc_path, layer='SOUNDG', LIST_AS_STRING=True, SPLIT_MULTIPOINT=True, ADD_SOUNDG_DEPTH=True)
depare_lyr = fiona.open(enc_path, layer='DEPARE', LIST_AS_STRING=True)

covr_geo = next(f for f in m_covr_lyr if f['properties']['CATCOV'] == 1)['geometry']
repr(covr_geo)[:200]

In [ ]:
len(list(soundg_lyr)) # 3415 - All soundings are independent features

## Écriture de l'emprise dans `data/covr.geojson`

In [ ]:
next(enumerate(soundg_lyr)) # Sample data

In [ ]:
covr_path = 'data/covr.geojson'
with fiona.open(covr_path, mode='w', driver='GeoJSON', schema={'properties':{}, 'geometry': 'Polygon'}) as cover_lyr:
    cover_lyr.write({'properties':{}, 'geometry': covr_geo})

# Lecture de la Bathy

## Réduction à la zone de l'ENC

In [ ]:
bathy_ds = gdal.Warp('data/bathy.tif', 'data/navd_bath_30m',
                        options = gdal.WarpOptions(
                                            format='GTiff', 
                                            cutlineDSName=covr_path,
                                            cutlineLayer='covr',
                                            cropToCutline=True,
                                            creationOptions=['COMPRESS=LZW'],
                                            dstNodata=np.nan))
del bathy_ds # ensure release and flush

In [ ]:
bathy_ds = rasterio.open('data/bathy.tif')

In [ ]:
assert len(bathy_ds.indexes) == 1
bathy_np = bathy_ds.read()[0]
bathy_ds.nodata

In [ ]:
bathy_np.shape # Northing, Easting

# PixelAccessor
PixelAccessor et tests

In [ ]:
bathy_ds.xy(0,0), bathy_ds.xy(2000,0), bathy_ds.xy(0, 2000)

In [ ]:
bathy_ds.index(328133.590626, 4722220.06), bathy_ds.index(388133.590626, 4722220.06), bathy_ds.index(328133.590626, 4662220.06)

In [ ]:
bathy_np[bathy_ds.index(415283.0,4717460.3)] # -125,9262

In [ ]:
class PixelAccessor: # Far Faster than pyproj
    def __init__(self, vector_lyr, raster_ds):
        geotransform = raster_ds.get_transform()
        assert geotransform[2] == 0
        assert geotransform[4] == 0

        c1 = osr.SpatialReference()
        c2 = osr.SpatialReference()
        osr.SpatialReference.ImportFromWkt(c1, vector_lyr.crs_wkt)
        osr.SpatialReference.ImportFromWkt(c2, raster_ds.crs.to_wkt())
        t12 = osr.CreateCoordinateTransformation(c1,c2)

        def latLonToPixel(lat, lon):
            Xp, Yp, Zp = t12.TransformPoint(lat, lon)
            P = math.floor((Xp - geotransform[0]) / geotransform[1])
            L = math.floor((Yp - geotransform[3]) / geotransform[5])
            return L, P
        
        def toPixel(geoPoint):
            return latLonToPixel(geoPoint[1], geoPoint[0])

        # Publish method Javascript style
        self.latLonToPixel = latLonToPixel
        self.toPixel = toPixel


In [ ]:
tw84 = PixelAccessor(depare_lyr, bathy_ds)

In [ ]:
# tw84.pixelToGeo(0,0), tw84.pixelToGeo(2000,0), tw84.pixelToGeo(0, 2000)

In [ ]:
tw84.toPixel((-71.09592403816357, 42.63312848595494)), tw84.toPixel((-71.07804409258947, 42.09312188970164)), tw84.toPixel((-70.36441180339453, 42.64417944703203))

In [ ]:
bathy_np[tw84.toPixel((-70.551710, 42.36582775))] # -88.0935

# Parcours d'isobathe

In [ ]:
for isobath in depare_lyr:
    print(isobath)